# 🗿 SwapFace Video Colab Extended Edition V2
[![Открыть в Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/self-destruction/SwapFace/blob/main/SwapFace_Video_Colab_Edition_V2.ipynb)

##### С помощью этого блокнота ты можешь <b>в пару кликов</b> сделать дип-фейк видео. Для этого нужна YouTube-ссылка на видео и фотография лица, на которое нужно заменить лицо. Для замены лица используется репозиторий [facefussion](https://github.com/facefusion/facefusion), но все возможные настройки перенесены в блокнот Google Colab.

In [1]:
#@title # Монтирование гугл диска
#@markdown ##### Гугл-диск используется только для сохранения итогового видео. Он не засоряется ничем другим.
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.utils import capture

with capture.capture_output() as cap:
    def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
    print("[0;33mПодключаю...")
    drive.mount('/content/drive')
    mainpth="MyDrive"
clear_output()

inf('\u2714 Готово','success', '300px')

Button(button_style='success', description='✔ Готово', disabled=True, layout=Layout(min_width='300px'), style=…

In [ ]:
# @title # Загружаем исходное лицо
#@markdown ##### Предоставьте изображение лица (.png, .jpg), на которое необходимо заменить лицо в клипе
#@markdown <font color=red>Здесь откроется проводник:</font>

from google.colab import files
import os
import matplotlib.pyplot as plt
from PIL import Image

img_folder = '/content/input/source' #@param {type:"string"}
!rm -rf {img_folder}
!mkdir -p {img_folder}
%cd {img_folder}
SOURCE_FACE_IMAGE = ''
TARGET_FACE_IMAGES = []

uploaded = files.upload()

for file_name in uploaded.keys():
  SOURCE_FACE_IMAGE = os.path.join(img_folder, file_name)

print(f"{SOURCE_FACE_IMAGE}")
plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(SOURCE_FACE_IMAGE))
plt.show()

In [ ]:
# @title #Установка facefussion
import ipywidgets as widgets
import os
from pathlib import PosixPath
import shutil

ROOT_DIR = '/content'
!mkdir -p {ROOT_DIR}/output

%cd {ROOT_DIR}
!git clone https://github.com/facefusion/facefusion -b next
SWAPFACE_REPO_PATH = f'{ROOT_DIR}/facefusion'
%cd {SWAPFACE_REPO_PATH}

!pip install -q moviepy yt_dlp ffmpeg ffmpeg-python virtualenv
!virtualenv venv
!source {SWAPFACE_REPO_PATH}/venv/bin/activate; python install.py --torch cuda --onnxruntime cuda


!sed -i 's/def analyse_frame(frame : Frame) -> bool:/def analyse_frame(frame : Frame) -> bool: True\ndef analyse_frame123(frame : Frame) -> bool:/g' {SWAPFACE_REPO_PATH}/facefusion/content_analyser.py

%cd {ROOT_DIR}
print('Готово! Продолжай дальше!')

In [ ]:
# @title # Скачиваем исходное видео (опционально)
# @markdown #### Этот шаг можно пропустить и вручную загрузить видео в любую папку.
#@markdown ##### Вставьте ссылку на Youtube:
import yt_dlp
import ffmpeg
import sys
from moviepy.editor import VideoFileClip
from pathlib import Path, PurePath
from re import sub

%cd {ROOT_DIR}
url = 'https://www.youtube.com/watch?v=n6P0SitRwy8' #@param {type:"string"}

input_download_path = f'{ROOT_DIR}/input/video'
video_download_format = 'mp4'

!rm -rf {input_download_path}
!mkdir -p {input_download_path}

video_opts = {
  'format': f'bestvideo[height<=1080][ext={video_download_format}]+bestaudio[ext=m4a]/best[height<=1080]',
  "outtmpl": f'{input_download_path}/%(title)s.%(ext)s',
}

def download_from_url(opts):
  with yt_dlp.YoutubeDL(opts) as ydl:
    ydl.download([url])

download_from_url(video_opts)

INPUT_VIDEO_FILE = str(next(Path(input_download_path).glob(f'*.{video_download_format}')))

new_name = sub(r'[^\w\-_\.]', '_', PurePath(INPUT_VIDEO_FILE).stem) + f'.{video_download_format}'
new_path = str(Path(PurePath(INPUT_VIDEO_FILE).parent).joinpath(new_name))

!mv "{INPUT_VIDEO_FILE}" "{new_path}"

INPUT_VIDEO_FILE = new_path

print(f"Видео готово: {INPUT_VIDEO_FILE}")

In [5]:
# @title # Выбираем видео
# @markdown ##### Предоставьте путь до папки, туда загрузите видео вручную (только один mp4-файл)
# @markdown ##### Оставь пустым, чтобы использовать скаченное на предыдущем шаге видео с YouTube
# @markdown ---
# @markdown ##### Путь до папки:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from base64 import b64encode
from pathlib import Path
from math import ceil
import os

input_video_folder = '' #@param {type:"string"}
if input_video_folder != '':
  INPUT_VIDEO_FILE = str(next(Path(input_video_folder).glob(f'*.mp4')))
print(f"Будет использоваться видео по пути: {INPUT_VIDEO_FILE}")

input_download_path = f'{ROOT_DIR}/input/video'
temp_input_dir = f'{input_download_path}/temp'

!rm -rf {temp_input_dir}
!mkdir -p {temp_input_dir}

video = VideoFileClip(INPUT_VIDEO_FILE)

# @markdown ##### Предпросмотр (без предпросмотра ячейка отработает моментально):
is_preview = False #@param {type:"boolean"}
# @markdown ##### Если включен предпросмотр, показать только один 30 секундный кусок (ячейка отработает быстрее):
is_only_one_clip = True #@param {type:"boolean"}
piece_seconds = 30
# Вычисляем количество 30-секундных клипов, которые можно получить из видео
num_clips = 0 if is_preview != True else ceil(video.duration / piece_seconds) if is_only_one_clip != True else 1
for i in range(num_clips):
    start = i * piece_seconds
    end = min((i + 1) * piece_seconds, video.duration)
    clip = video.subclip(start, end).resize(height=420)
    temp_clip_file = os.path.join(temp_input_dir, f"clip_{i}.mp4")
    clip.write_videofile(temp_clip_file, logger=None, threads=64)
    mp4 = open(temp_clip_file,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video id="my_video" width=420 controls>
      <source src="%s" type="video/mp4">
    </video>
    <script>
    document.getElementById('my_video').volume = 0.2;
    </script>
    """ % data_url))

Будет использоваться видео по пути: /content/input/video/Nirvana_-_Heart-Shaped_Box__Official_Music_Video_.mp4


In [ ]:
from PIL import Image
import os
from pathlib import Path
import subprocess

debug_mode = True

#@title # Тестовый прогон
# @markdown ### <font color="yellow">Для применения настроек необходимо запустить ячейку</font>
# @markdown ### Выберите кадр из видео с помощью слайдера (левый край - начало, правый край - конец):
slider_value = 726838 # @param {type:"slider", min:1, max:1000000, step:1}

# @markdown ---
# @markdown ### <b>Настройки дип-фейка:</b>
# @markdown ##### Улучшение лица после замены:
face_enhancer_model = 'restoreformer' # @param ["none", "codeformer", "gfpgan_1.2", "gfpgan_1.3", "gfpgan_1.4", "gpen_bfr_512", "restoreformer"]
face_enhancer_blend = 100 # @param {type:"slider", min:0, max:100, step:1}
face_enhancer_blend = str(face_enhancer_blend)
# @markdown ##### Модель замены лица (<font color="grey">inswapper_128</font> - обычная, <font color="grey">inswapper_128_fp16</font> - быстрая):
face_swapper_model = 'inswapper_128_fp16' # @param ["inswapper_128", "inswapper_128_fp16", "simswap_256", "simswap_512_unofficial"]
# @markdown ##### Улучшение финального кадра:
frame_enhancer_model = 'none' # @param ["none", "real_esrgan_x2plus", "real_esrgan_x4plus", "real_esrnet_x4plus"]
frame_enhancer_blend = 100 # @param {type:"slider", min:0, max:100, step:1}
frame_enhancer_blend = str(frame_enhancer_blend)
# @markdown ##### Тип замены (reference - лицо с текущего выбранного кадра, many - несколько лиц):
face_selector_mode = 'reference' # @param ["reference", "many"]
# @markdown ##### Условия выбора лица (только если выбран тип замены "many"):
face_analyser_gender = 'none' # @param ["none", "male", "female"]
if face_analyser_gender != 'none':
  face_analyser_gender = f"--face-analyser-gender {face_analyser_gender} "
else:
  face_analyser_gender = ''
face_analyser_age = 'none' # @param ["none", "child", "teen", "adult", "senior"]
if face_analyser_age != 'none':
  face_analyser_age = f"--face-analyser-age {face_analyser_age} "
else:
  face_analyser_age = ''
face_detector_model = 'retinaface' # @param ["retinaface", "yunet"]

temp_dir = '/content/input/temp'
temp_file_name = 'temp.jpg'
temp_file = os.path.join(temp_dir, temp_file_name)

!rm -rf {temp_dir}
!mkdir -p {temp_dir}

%cd {SWAPFACE_REPO_PATH}

# получаем количество кадров в видео
clip = VideoFileClip(INPUT_VIDEO_FILE)
frame_count = clip.fps * clip.duration
frame = int((slider_value / 1000000) * frame_count)
time_sec = frame / clip.fps
print(f"Кадр {frame}, секунда видео {int(time_sec)}")

# получаем исходный кадр из видео
cmd = (
 f"import os, cv2;"
 f"cap = cv2.VideoCapture('{INPUT_VIDEO_FILE}', cv2.CAP_FFMPEG);"
 f"cap.set(cv2.CAP_PROP_POS_FRAMES, min({frame}, cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1));"
 f"valid_frame, frame = cap.read();"
 f"cap.release();"
 f"frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if valid_frame else None;"
 f"print('frame is None') if frame is None else None;"
 f"frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if frame is not None else None;"
 f"print('Failed to save image') if not cv2.imwrite('{temp_file}', frame) else None;"
)

!source {SWAPFACE_REPO_PATH}/venv/bin/activate; python -c "{cmd}"

plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(temp_file))
plt.show()

# ---

OUTPUT_PATH = f'{ROOT_DIR}/output'
output_name = 'temp_output'
temp_ext = '.jpg'
# _, temp_ext = os.path.splitext(SOURCE_FACE_IMAGE)
temp_processed_file = os.path.join(OUTPUT_PATH, f"{output_name}{temp_ext}")

!rm -rf {temp_processed_file}

face_swapper_processor = 'face_swapper'
face_enhancer_processor = 'face_enhancer'
frame_enhancer_processor = 'frame_enhancer'

frame_processors = face_swapper_processor

if face_enhancer_model != 'none':
    frame_processors = frame_processors + ' ' + face_enhancer_processor
else:
    face_enhancer_model = 'codeformer'
if frame_enhancer_model != 'none':
    frame_processors = frame_processors + ' ' + frame_enhancer_processor
else:
    frame_enhancer_model = 'real_esrgan_x2plus'

cmd = (
    f"--source {Path(SOURCE_FACE_IMAGE)} "
    f"--target {temp_file} "
    f"--output {temp_processed_file} "
    f"--headless "
    f"--execution-providers cuda "
    f"--execution-thread-count 128 "
    f"--execution-queue-count 16 "
    f"--max-memory 128 "
    f"--face-analyser-order left-right "
    f"{face_analyser_age}"
    f"{face_analyser_gender}"
    f"--face-detector-model {face_detector_model} "
    f"--face-detector-size 640x640 "
    f"--face-detector-score 0.5 "
    f"--face-selector-mode {face_selector_mode} "
    f"--reference-face-position 0 "
    f"--reference-face-distance 1.5 "
    f"--reference-frame-number {frame} "
    f"--trim-frame-start 0 "
    f"--trim-frame-end 0 "
    f"--temp-frame-format jpg "
    f"--temp-frame-quality 100 "
    f"--output-image-quality 100 "
    f"--output-video-encoder libx264 "
    f"--output-video-quality 100 "
    f"--keep-fps "
    f"--output-image-quality 100 "
    f"--frame-processors {frame_processors} "
    f"--face-enhancer-model {face_enhancer_model} "
    f"--face-enhancer-blend {face_enhancer_blend} "
    f"--face-swapper-model {face_swapper_model} "
    f"--frame-enhancer-model {frame_enhancer_model} "
    f"--frame-enhancer-blend {frame_enhancer_blend} "
    f"--ui-layouts default"
)

if debug_mode == True:
    print(cmd)

!source {SWAPFACE_REPO_PATH}/venv/bin/activate; python run.py {cmd}

plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(temp_processed_file))
plt.show()

In [ ]:
from IPython.display import display, HTML
from pathlib import Path
from moviepy.editor import VideoFileClip
from base64 import b64encode
from math import ceil
import os

#@title # Боевой прогон
# @markdown ### Делаем дипфейк видео
# @markdown *Настройки берутся из тестового прогона, поэтому перед запуском проверьте их на ячейке "Тестовый прогон"*

# @markdown ### Прогнать тестовые N секунд (начиная с ранее выбранного кадра)
is_test_launch = False # @param {type:"boolean"}
N = 3 # @param {type:"slider", min:1, max:30, step:1}

%cd {SWAPFACE_REPO_PATH}

GOOGLE_DRIVE_OUTPUT_FOLDER = f'{ROOT_DIR}/drive/MyDrive/SwapFace/output'
OUTPUT_VIDEO_PATH = f'{ROOT_DIR}/output'
output_video_name = os.path.splitext(os.path.basename(INPUT_VIDEO_FILE))[0]
OUTPUT_VIDEO = os.path.join(GOOGLE_DRIVE_OUTPUT_FOLDER, f"{output_video_name}.mp4")
video_input = INPUT_VIDEO_FILE
temp_output_dir = f'{OUTPUT_PATH}/temp'

!mkdir -p {GOOGLE_DRIVE_OUTPUT_FOLDER}
!mkdir -p {OUTPUT_VIDEO_PATH}
!rm -rf {temp_output_dir}
!mkdir -p {temp_output_dir}

cmd = (
    f"--source {Path(SOURCE_FACE_IMAGE)} "
    f"--target {video_input} "
    f"--output {OUTPUT_VIDEO} "
    f"--headless "
    f"--execution-providers cuda "
    f"--execution-thread-count 64 "
    f"--execution-queue-count 16 "
    f"--max-memory 16 "
    f"--face-recognition many "
    f"--face-analyser-direction left-right "
    f"--face-analyser-age adult "
    f"--face-analyser-gender female "
    f"--reference-face-position 0 "
    f"--reference-face-distance 1.5 "
    f"--reference-frame-number {frame} "
    # f"--trim-frame-start 0 "
    # f"--trim-frame-end 0 "
    f"--temp-frame-format jpg "
    f"--temp-frame-quality 80 "
    f"--output-image-quality 100 "
    f"--output-video-encoder libx264 "
    f"--output-video-quality 100 "
    f"--keep-fps "
    f"--output-image-quality 100 "
    f"--frame-processors {frame_processors} "
    f"--face-enhancer-model {face_enhancer_model} "
    f"--face-enhancer-blend {face_enhancer_blend} "
    f"--face-swapper-model {face_swapper_model} "
    f"--frame-enhancer-model {frame_enhancer_model} "
    f"--frame-enhancer-blend {frame_enhancer_blend} "
    f"--ui-layouts default"
)

if is_test_launch == True:
  temp_videofile_name = 'temp_video.mp4'
  OUTPUT_VIDEO = os.path.join(GOOGLE_DRIVE_OUTPUT_FOLDER, temp_videofile_name)

  clip = VideoFileClip(video_input)
  start_time = frame / clip.fps
  frame_finish = int(frame + N * clip.fps)
  cmd = (
      f"--source {Path(SOURCE_FACE_IMAGE)} "
      f"--target {video_input} "
      f"--output {OUTPUT_VIDEO} "
      f"--headless "
      f"--execution-providers cuda "
      f"--execution-thread-count 64 "
      f"--execution-queue-count 16 "
      f"--max-memory 16 "
      f"--face-recognition many "
      f"--face-analyser-direction left-right "
      f"--face-analyser-age adult "
      f"--face-analyser-gender female "
      f"--reference-face-position 0 "
      f"--reference-face-distance 1.5 "
      f"--reference-frame-number {frame} "
      f"--trim-frame-start {frame} "
      f"--trim-frame-end {frame_finish} "
      f"--temp-frame-format jpg "
      f"--temp-frame-quality 80 "
      f"--output-image-quality 100 "
      f"--output-video-encoder libx264 "
      f"--output-video-quality 100 "
      f"--keep-fps "
      f"--output-image-quality 100 "
      f"--frame-processors {frame_processors} "
      f"--face-enhancer-model {face_enhancer_model} "
      f"--face-enhancer-blend {face_enhancer_blend} "
      f"--face-swapper-model {face_swapper_model} "
      f"--frame-enhancer-model {frame_enhancer_model} "
      f"--frame-enhancer-blend {frame_enhancer_blend} "
      f"--ui-layouts default"
  )

if debug_mode == True:
    !echo "{cmd}"

!source {SWAPFACE_REPO_PATH}/venv/bin/activate; python run.py {cmd}

NEW_OUTPUT_VIDEO = os.path.join(GOOGLE_DRIVE_OUTPUT_FOLDER, os.path.basename(OUTPUT_VIDEO))
!mv {OUTPUT_VIDEO} {GOOGLE_DRIVE_OUTPUT_FOLDER}

print(f"Видео готово: {NEW_OUTPUT_VIDEO}")

video = VideoFileClip(NEW_OUTPUT_VIDEO)

# @markdown ##### В превью показать только один 30 секундный кусок (ячейка отработает быстрее):
is_only_one_clip = True #@param {type:"boolean"}
piece_seconds = 10
# Вычисляем количество 30-секундных клипов, которые можно получить из видео
num_clips = ceil(video.duration / piece_seconds) if is_only_one_clip != True else 1
for i in range(num_clips):
    start = i * piece_seconds
    end = min((i + 1) * piece_seconds, video.duration)
    clip = video.subclip(start, end).resize(height=420)
    temp_clip_file = os.path.join(temp_output_dir, f"clip_{i}.mp4")
    print(f"Вывожу кусок на превью: {temp_clip_file}")
    clip.write_videofile(temp_clip_file, logger=None, threads=32)
    mp4 = open(temp_clip_file,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video id="my_video" width=420 controls>
      <source src="%s" type="video/mp4">
    </video>
    <script>
    document.getElementById('my_video').volume = 0.2;
    </script>
    """ % data_url))